Pseudobulk analysis of resolving vs non-resolving RPRA samples from [Bailey et al. bioRxiv 2023](https://www.biorxiv.org/content/10.1101/2023.07.30.551145v1.full#F1).

Resolving samples come from subjects whose area of normal lung increased between two visits.

Non-resolving are the opposite, transplant samples are not included.

In [1]:
import os
import pathlib
import anndata
import scanpy as sc
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

/home/kjs6397/.conda/envs/pasc_revisions_venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
OUT_DIR = "../../../data/22deg-analysis/bailey_resolv_vs_nonresolv_v2/pseudobulk/input"

In [3]:
if not os.path.exists(OUT_DIR):
    os.makedirs(OUT_DIR)

## Load dataset
Use object with raw counts from preprint.

In [4]:
adata = sc.read_h5ad('/projects/b1038/Pulmonary/cpuritz/PASC/data/01BAL/raw/adata_raw_final.h5ad')

### Rename genes to remove GRCh38 prefix

In [5]:
adata.var['gene_ids'] = adata.var['gene_ids'].str.replace('GRCh38_', '')

In [6]:
adata.obs

,Library_ID,Study_ID,is_RPRA,Status,cell_type,n_genes_by_counts,total_counts,pct_counts_in_top_50_genes,pct_counts_in_top_100_genes,pct_counts_in_top_200_genes,pct_counts_in_top_500_genes,total_counts_mito,pct_counts_mito
AAACCTGAGACCACGA-1-Library_5D_4720,Library_5D_4720,RPRA02,True,RPRA,TRAM-6,3938,19553.0,36.367821,46.816345,58.078044,70.899606,619.0,3.165755
AAACCTGAGGCAAAGA-1-Library_5D_4720,Library_5D_4720,RPRA02,True,RPRA,MoAM-2,2486,6599.0,28.170935,37.414760,48.386119,63.812699,331.0,5.015912
AAACCTGAGGCGCTCT-1-Library_5D_4720,Library_5D_4720,RPRA02,True,RPRA,MoAM-4,2168,6862.0,34.071699,44.622559,56.368406,71.466045,271.0,3.949286
AAACCTGAGTAATCCC-1-Library_5D_4720,Library_5D_4720,RPRA02,True,RPRA,MoAM-4,2083,6794.0,36.694142,47.762732,59.640860,73.226376,242.0,3.561966
AAACCTGAGTCATGCT-1-Library_5D_4720,Library_5D_4720,RPRA02,True,RPRA,Perivascular macrophages,4303,25935.0,37.624831,47.507230,58.261037,71.305186,433.0,1.669559
...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGCGCGTTAAGAAC-1-Library_1X_2236,Library_1X_2236,HV06,False,Healthy,CD4 T cells-1,1128,2257.0,32.831192,44.395215,55.560479,72.175454,104.0,4.607886
TTTGGTTCATCGTCGG-1-Library_1X_2236,Library_1X_2236,HV06,False,Healthy,MoAM-4,4207,18687.0,32.990849,42.232568,52.426821,65.874672,290.0,1.551881
TTTGGTTGTTCCAACA-1-Library_1X_2236,Library_1X_2236,HV06,False,Healthy,TRAM-2,3627,24140.0,42.887324,52.394366,63.235294,76.122618,0.0,0.000000
TTTGTCAAGCTATGCT-1-Library_1X_2236,Library_1X_2236,HV06,False,Healthy,TRAM-4,3444,12490.0,33.795036,42.866293,52.481986,65.700560,471.0,3.771017


### Subset on resolving RPRA samples
Samples from subjects whose area of normal lung increased between two visits.

In [7]:
resolving_raw = adata[adata.obs.Study_ID.isin(['RPRA01', 'RPRA02', 'RPRA03', 'RPRA04', 'RPRA07',
                                                'RPRA08', 'RPRA09', 'RPRA10', 'RPRA14', 'RPRA15',
                                                'RPRA16', 'RPRA19', 'RPRA21', 'RPRA22', 'RPRA25',
                                                'RPRA26', 'RPRA27', 'RPRA30', 'RPRA31', 'RPRA32',
                                                'RPRA33', 'RPRA29'
                                               ])]

### Subset on non-resolving PASC samples
Samples from subjects whose area of normal lung decreased between two visits.

In [8]:
non_resolving_raw = adata[adata.obs.Study_ID.isin(['RPRA05', 'RPRA06', 'RPRA11', 'RPRA23', 
                                                         'RPRA24', 'RPRA34', 'RPRA17'
                                               ])]

### Ensure metadata looks the same between datasets

In [9]:
resolving_raw.obs = resolving_raw.obs.rename(columns={'Study_ID': 'sample'})

In [10]:
non_resolving_raw.obs = non_resolving_raw.obs.rename(columns={'Study_ID': 'sample'})

In [11]:
common_obs = list(set(resolving_raw.obs.columns) & set(non_resolving_raw.obs.columns))

In [12]:
resolving_raw.obs = resolving_raw.obs[common_obs]
non_resolving_raw.obs = non_resolving_raw.obs[common_obs]

### Identify dataset by index

In [13]:
resolving_raw.obs.index = 'PASC_' + resolving_raw.obs.index
non_resolving_raw.obs.index = 'PASC_' + non_resolving_raw.obs.index

### Create new column called "Status"
This is the DESeq2 comparisons

In [14]:
resolving_raw.obs['Status'] = 'RPRA resolving'
non_resolving_raw.obs['Status'] = 'RPRA non-resolving'

### Create new column called "Study"

In [15]:
resolving_raw.obs['Study'] = 'Bailey'
non_resolving_raw.obs['Study'] = 'Bailey'

## Concatenate both objects
Subset on common genes - mitochondrial genes, ribosomal genes, and meaningless transcripts have not yet been filtered out of Reyfman dataset

In [16]:
adata = sc.concat([resolving_raw, non_resolving_raw], join='inner')

In [17]:
common_genes = adata.var.index.tolist()

## Remove meaningless genes 
### Remove transcripts with 'AC', 'AP', 'AL', 'AF', etc. prefix from list of genes. Also remove ribosomal genes and mitochondrial genes. Also remove genes unique to 3' chemistry and 5' chemistry.
These are "genes" are actually transcripts with prefixes like ACXXXXX, APXXXXXX, and ALXXXXXX. Since these are not biologically significant genes, it makes sense to exclude them. However, we don't expect to find them expressed since they are uniquely detected in 3' chemistry.

In [18]:
# specify the prefix pattern
prefixes_to_remove = ['AC', 'AL', 'AP', 'AF', 'AD', 'BX', 'CR', 'FP', 'KF']

# specify tlhe pattern to match genes to remove
pattern_to_remove = '|'.join([f'{prefix}\d{{6}}\.\d' for prefix in prefixes_to_remove] + ['Z\d{5}\.\d', 'U\d{5}\.\d'])

In [19]:
three_p_only = pd.read_csv('../../../data/21scArches/unique_genes_chem/three_prime_only.csv', index_col=0)

In [20]:
three_p_only = three_p_only.index.tolist()
# Keep SARS genes
three_p_only = [gene for gene in three_p_only if not gene.startswith("SARS2")]

In [21]:
five_p_only = pd.read_csv('../../../data/21scArches/unique_genes_chem/five_prime_only.csv', index_col=0)

In [22]:
five_p_only = five_p_only.index.tolist()
# Keep SARS genes
five_p_only = [gene for gene in five_p_only if not gene.startswith("SARS2")]

In [23]:
# Create boolean masks for both conditions
pattern_mask = adata.var.index.str.contains(pattern_to_remove, regex=True)
ribo_mito_mask = adata.var.index.str.startswith(("RPS", "RPL", "MT-"))
three_p_mask = adata.var.index.isin(three_p_only)
five_p_mask = adata.var.index.isin(five_p_only)

# Create a combined mask 
combined_mask = (pattern_mask | ribo_mito_mask | three_p_mask | five_p_mask)

In [24]:
print(pattern_mask.sum())
print(ribo_mito_mask.sum())
print(three_p_mask.sum())
print(five_p_mask.sum())
print(combined_mask.sum())

12510
116
2480
1150
14377


In [25]:
# Create a new AnnData object with the filtered data
adata = anndata.AnnData(
    X=adata.X[:, ~combined_mask],
    obs=adata.obs,
    var=adata.var[~combined_mask],
    obsm=adata.obsm,
    uns=adata.uns
)

In [26]:
adata.layers["counts"] = adata.X.copy()

In [27]:
adata

AnnData object with n_obs × n_vars = 169741 × 22236
    obs: 'Library_ID', 'is_RPRA', 'pct_counts_in_top_50_genes', 'Status', 'sample', 'n_genes_by_counts', 'pct_counts_in_top_500_genes', 'pct_counts_in_top_200_genes', 'total_counts_mito', 'total_counts', 'cell_type', 'pct_counts_mito', 'pct_counts_in_top_100_genes', 'Study'
    layers: 'counts'

### Sanity check: ensure there are no genes that are uniquely detected in 3' or 5' chemistry

In [28]:
unique_genes = three_p_only + five_p_only

In [29]:
common_genes = adata.var.index.tolist()

In [30]:
len(list(set(unique_genes) & set(common_genes)))

0

## Write pseudobulk counts (gene sums) and number of cells for each sample

In [31]:
CUTOFF = 50
for ct in adata.obs.cell_type.unique():
    ct_slug = ct.lower().replace(' ', '_').replace('/', '_')
    cells = adata.obs.index[adata.obs.cell_type == ct]
    adata.obs['sample'] = [i.replace(' ','_') for i in adata.obs['sample']]
    adata.obs['sample'] = [i.replace('-','_') for i in adata.obs['sample']]
    meta = adata.obs.loc[cells, ["sample", "Status", "Study"]].drop_duplicates()
    samples = adata.obs["sample"].unique()
    sample_values = [] # gene sum for sample in this cell type
    sample_ncells = [] # number of cells expressing gene for sample in this cell type
    filtered_samples = [] # samples that pass cutoff filter
    n_cells = []
    for s in samples:
        s_cells = adata.obs_names.isin(cells) & (adata.obs["sample"] == s) # cells in particular sample
        if s_cells.sum() >= CUTOFF:
            sample_values.append(adata.X[s_cells, :].sum(axis=0).A[0]) # gene sum across all cells in sample
            sample_ncells.append((adata.X[s_cells, :] > 0).sum(axis=0).A[0]) # number of cells expressing gene
            filtered_samples.append(s)
            n_cells.append(s_cells.sum())
            
    sample_values = pd.DataFrame(sample_values, index=filtered_samples, columns=adata.var_names).T
    sample_ncells = pd.DataFrame(sample_ncells, index=filtered_samples, columns=adata.var_names).T

    fname = f"{OUT_DIR}/{ct_slug}.txt"
    sample_values.to_csv(fname, sep="\t")
    
    fname = f"{OUT_DIR}/{ct_slug}-n_cells.txt"
    sample_ncells.to_csv(fname, sep="\t")
    
    fname = f"{OUT_DIR}/{ct_slug}-meta.csv"
    meta = meta.loc[meta["sample"].isin(filtered_samples), :]
    meta["n_cells"] = n_cells
    meta.to_csv(fname)
    print(f"{ct} done, {len(filtered_samples)}/{len(samples)}")

TRAM-6 done, 18/20
MoAM-2 done, 19/20
MoAM-4 done, 17/20
Perivascular macrophages done, 19/20
TRAM-4 done, 16/20
TRAM-3 done, 17/20
Migratory DC done, 3/20
TRAM-1 done, 17/20
TRAM-2 done, 14/20
CD8 T cells-3 done, 17/20
MoAM-3 done, 18/20
Proliferating macrophages done, 18/20
TRAM-5 done, 19/20
pDC done, 4/20
TRAM-7 done, 13/20
DC2 done, 18/20
CD8 T cells-1 done, 15/20
CD4 T cells-1 done, 17/20
gdT cells and NK cells done, 14/20
Tregs done, 12/20
CD8 T cells-2 done, 16/20
CD4 T cells-2 done, 16/20
MoAM-1 done, 19/20
DC1 done, 5/20
Proliferating T cells done, 5/20
Mast cells done, 4/20
Monocytes-2 done, 5/20
B cells done, 7/20
Monocytes-1 done, 2/20
Plasma cells done, 1/20
Epithelial cells done, 0/20
SARS-CoV-2 done, 0/20


### Save file

In [32]:
adata.write(f'{OUT_DIR}/bailey_resolv_vs_nonresolv_v2_pseudobulk.h5ad')